In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from tqdm import tqdm_notebook

In [10]:
%run data.ipynb
#from data import *
%run image_model.ipynb
#from model import *
%run sequence_model.ipynb
#from model import *
%run config.ipynb
#from config import *
%run utils.ipynb
#from utils import *

In [ ]:
# generate data loader
train_loader = get_loader(train=True, image=IMAGE_MODE, sequence=SEQUENCE_MODE)
val_loader = get_loader(val=True, image=IMAGE_MODE, sequence=SEQUENCE_MODE)
test_loader = get_loader(test=True, image=IMAGE_MODE, sequence=SEQUENCE_MODE)

In [ ]:
def main():
    # model
    Net = ImageNet if IMAGE_MODE else SequenceNet
    model = Net().cuda()
    
    # optimizer and scheduler
    optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=initial_lr)
    scheduler = lr_scheduler.ExponentialLR(optimizer, 0.5**(1 / lr_halflife))
    start_epoch = 0
    
    for i in range(num_epoch):
        #'Training epoch: '
        train_ac = train(model, train_loader, optimizer, scheduler, i, train=True)
        #'Evaluation epoch: '
        val_ac = train(model, val_loader, optimizer, scheduler, i, train=False)
        
        #save model
        results = {
            'epoch':i,
            'weights':model.state_dict(),
            'training accuracy': train_ac,
            'eval accuracy':val_ac,
        }
        torch.save(results, model_name + '_' + str(i) + '.pth')
        

In [ ]:
def train(model, loader, optimizer, scheduler, epoch, train=False):
    if train:
        model.train()
    else:
        model.eval()
    
     # container
    output_container = []
    loss_container = []
    
    loader = tqdm_notebook(loader, desc='Training epoch '+str(epoch) if train else 'Eval epoch '+str(epoch))
    for v, l in loader:
        var_params = {
            'requires_grad': False,
        }
        v = Variable(v.cuda(async=True), **var_params)
        l = Variable(l.cuda(async=True), **var_params)
        
        out = model(v)
        if train:
            loss = F.cross_entropy(out, l)
            acc = (l == out.max(1)[1]).float().mean()
            output_container.append(float(acc.detach().data))
            loss_container.append(float(loss.detach().data))
            scheduler.step()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        else:
            acc = (l == out.max(1)[1]).float().mean()
            output_container.append(float(acc))
            
        # visualization
        loader.set_postfix(loss=list_mean(loss_container), acc=list_mean(output_container))
    
    return list_mean(output_container)

In [ ]:
main()